# Ch4 - 1. merge

### merge가 필요한 상황  

효율적인 데이터 베이스 관리를 위해, 잘 정제된 데이터일지라도 데이터가 키 변수를 기준으로 나뉘어 저장되는 경우가 매우 흔함.  
SQL에서는 JOIN을 이용, python에서는 pd.merge를 이용함.  

### pd.merge
키 변수를 기준으로 두 개의 데이터 프레임을 병함(join)하는 함수.  

주요 입력 파라미터  
> `left` : 통합 대상 데이터프레임1  
`right` : 통합 대상 데이터프레임2  
`on` : 통합 기준 key 변수 및 변수 리스트(입력을 하지 않으면 이름이 같은 변수를 key로 식별, 일반적으로는 반드시 입력을 해주어야 한다.)  
`left_on` : 데이터프레임1의 key 변수 및 변수 리스트  
`right_on` : 데이터프레임2의 key 변수 및 변수 리스트  
`left_index` : 데이터프레임1의 인덱스를 key변수로 사용할 지 여부  
`right_index` : 데이터프레임2의 인덱스를 key변수로 사용할 지 여부  



In [1]:
import os
import pandas as pd
os.chdir(r"/Users/Angela/Desktop/과속대학쿠쿠루/1. 데이터 핸들링/데이터")

## merge


불러올때 Tip.  
`OSError: Initializing from file failed`가 발생하면 read_csv 변수에 `engine = "python"` 을 설정하면 피할 수 있음 (단, 속도 저하 문제가 있음)


merge를 하면 순서는 left 데이터 기준으로 붙게 되어있다.  
엑셀에 비교하면 Vlookup과 비슷함.  

In [3]:
df1 = pd.read_csv("merge_실습_employee_group.csv")
df2 = pd.read_csv("merge_실습_employee_hire_date.csv")

In [4]:
df1.head()

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [5]:
df2.head()

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [7]:
# 컬럼명이 같아서 키를 employee로 인식한다. employee를 기준으로 merge됨.

merged_df = pd.merge(df1, df2)
merged_df.head()

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### on을 사용하는 경우

In [6]:
merged_df = pd.merge(df1, df2, on = 'employee')
merged_df.head()

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [8]:
# 리스트로도 받을 수 있기 때문에 감싸줄 수 있다. 
merged_df = pd.merge(df1, df2, on = ["employee"])
merged_df.head()

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


## left_on, right_on을 사용한 데이터 통합

묶어줄 기준이 될 컬럼의 이름이 다를 경우 지정해준다.  
이때 이 컬럼 모두 살아있게 되므로 하나를 제거해주어야 한다.  

In [9]:
df1 = pd.read_csv("merge_실습_employee_group.csv")
df2 = pd.read_csv("merge_실습_name_hire_date.csv")

In [10]:
df1.head()

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [11]:
df2.head()

,hire_date,name
0,2004,Lisa
1,2008,Bob
2,2012,Jake
3,2014,Sue


In [12]:
# 컬럼명이 다를 경우 left_on, right_on을 사용한다.
merged_df = pd.merge(df1, df2, left_on = 'employee', right_on = 'name')
merged_df.head()

,employee,group,hire_date,name
0,Bob,Accounting,2008,Bob
1,Jake,Engineering,2012,Jake
2,Lisa,Engineering,2004,Lisa
3,Sue,HR,2014,Sue


'name'이라는 인덱스를 axis = 1, 즉 열에서 찾겠다는 의미. axis는 자주 헷갈리는 기분.  

In [13]:
merged_df.drop('name', axis = 1, inplace = True) 
merged_df.head()

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


## index를 사용한 데이터 통합

`left_index`, `right_index`를 사용하는 경우.  

붙여야할 기준이 컬럼이 아니라 인덱스일 경우.  
키로 하는 컬럼은 `_on = ''`, 키로 하는 인덱스는 `_index = True` 로 적는다고 생각하자.  

index는 컬럼 취급을 하지 않기 때문에 새로 컬럼이 생기지 않아서 지울 필요가 없다.  



In [14]:
df1 = pd.read_csv("merge_실습_employee_group.csv")
df2 = pd.read_csv("merge_실습_name_hire_date.csv", index_col = "name")

In [15]:
df2.head()

,hire_date
name,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [16]:
merged_df = pd.merge(df1, df2, left_on = 'employee', right_index = True)
merged_df.head()

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### left와 right 상황을 바꾸었을 때 1

In [17]:
df1 = pd.read_csv("merge_실습_employee_group.csv", index_col = 'employee')
df2 = pd.read_csv("merge_실습_name_hire_date.csv")

In [18]:
df1

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [20]:
merged_df = pd.merge(df1, df2, left_index = True, right_on = 'name')
merged_df

,group,hire_date,name
1,Accounting,2008,Bob
2,Engineering,2012,Jake
0,Engineering,2004,Lisa
3,HR,2014,Sue


### left와 right 상황을 바꾸었을 때 2

In [21]:
merged_df = pd.merge(df2, df1, right_index = True, left_on = 'name')
merged_df

,hire_date,name,group
0,2004,Lisa,Engineering
1,2008,Bob,Accounting
2,2012,Jake,Engineering
3,2014,Sue,HR
